In [1]:
from dataclasses import dataclass
import pandas as pd
%matplotlib inline 
from sensitivity import SensitivityAnalyzer

In [2]:
#Normal Economy
@dataclass
class ModelInputs:
    starting_salary: int = 60000
    promos_every_n_years: int = 5
    cost_of_living_raised: float = 0.02
    promo_raise: float =0.15
    savings_rate: float =0.25
    interest_rate: float =0.05
    desired_cash: int = 1500000
model_data = ModelInputs()
model_data

ModelInputs(starting_salary=60000, promos_every_n_years=5, cost_of_living_raised=0.02, promo_raise=0.15, savings_rate=0.25, interest_rate=0.05, desired_cash=1500000)

In [3]:
#bad Economy
bad_economy_data=ModelInputs(
    starting_salary=40000,
    promos_every_n_years=8,
    cost_of_living_raised=0.01,
    promo_raise=0.07,
    savings_rate=0.15,
    interest_rate=0.03
    )

In [4]:
#bad Economy
good_economy_data=ModelInputs(
    starting_salary=70000,
    promos_every_n_years=4,
    cost_of_living_raised=0.03,
    promo_raise=0.02,
    savings_rate=0.35,
    interest_rate=0.06
    )
cases={
    'Recession':bad_economy_data,
    'Normal':model_data,
    'Expansion':good_economy_data
}

In [5]:
def years_to_retirement(model_data: ModelInputs,print_output=False):
    
    # starting with no cash saved
    data=model_data
    prior_wealth = 0  
    wealth = 0
    
    year = 0  # will become 1 on first loop
    
    def wealth_at_year(data,year,prior_wealth):
        cash_saved = cash_saved_during_year(data,year)
        wealth = prior_wealth*(1 + data.interest_rate) + cash_saved
        return wealth
    
    def cash_saved_during_year(model_data,year):
        salary= salary_at_year(model_data,year)
        cash_saved=salary*model_data.savings_rate
        return cash_saved
    
    def salary_at_year(data: ModelInputs, year):
        # Every n years we have a promotion, so dividing the years and taking out the decimals gets the number of promotions
        num_promos = int(year / data.promos_every_n_years)
        
        # This is the formula above implemented in Python
        salary_t = data.starting_salary * (1 + data.cost_of_living_raised) ** year * (1 + data.promo_raise) ** num_promos
        return salary_t

    if print_output:
        print('Wealths over time:') # \n makes a blank line in the output.
    while wealth < data.desired_cash:
        year = year + 1
        wealth = wealth_at_year(model_data, year, prior_wealth)
        if print_output:
            print(f'The wealth at year {year} is ${wealth:,.0f}.')
        # Set next year's prior wealth to this year's wealth
        prior_wealth = wealth
        
    # Now we have exited the while loop, so wealth must be >= desired_cash. Whatever last year was set
    # is the years to retirement.
    if print_output:
        print(f'\nRetirement:\nIt will take {year} years to retire.')  # \n makes a blank line in the output.
    return year
years = years_to_retirement(model_data,print_output=True)

Wealths over time:
The wealth at year 1 is $15,300.
The wealth at year 2 is $31,671.
The wealth at year 3 is $49,173.
The wealth at year 4 is $67,868.
The wealth at year 5 is $90,307.
The wealth at year 6 is $114,248.
The wealth at year 7 is $139,775.
The wealth at year 8 is $166,975.
The wealth at year 9 is $195,939.
The wealth at year 10 is $229,918.
The wealth at year 11 is $266,080.
The wealth at year 12 is $304,542.
The wealth at year 13 is $345,431.
The wealth at year 14 is $388,878.
The wealth at year 15 is $439,025.
The wealth at year 16 is $492,294.
The wealth at year 17 is $548,853.
The wealth at year 18 is $608,878.
The wealth at year 19 is $672,557.
The wealth at year 20 is $745,168.
The wealth at year 21 is $822,190.
The wealth at year 22 is $903,859.
The wealth at year 23 is $990,422.
The wealth at year 24 is $1,082,140.
The wealth at year 25 is $1,185,745.
The wealth at year 26 is $1,295,520.
The wealth at year 27 is $1,411,793.
The wealth at year 28 is $1,534,910.

Reti

In [6]:
for case_type,case_inputs in cases.items():
    print(case_inputs)
    print(case_type)

ModelInputs(starting_salary=40000, promos_every_n_years=8, cost_of_living_raised=0.01, promo_raise=0.07, savings_rate=0.15, interest_rate=0.03, desired_cash=1500000)
Recession
ModelInputs(starting_salary=60000, promos_every_n_years=5, cost_of_living_raised=0.02, promo_raise=0.15, savings_rate=0.25, interest_rate=0.05, desired_cash=1500000)
Normal
ModelInputs(starting_salary=70000, promos_every_n_years=4, cost_of_living_raised=0.03, promo_raise=0.02, savings_rate=0.35, interest_rate=0.06, desired_cash=1500000)
Expansion


In [7]:
for case_type,case_inputs in cases.items():
    ytr=years_to_retirement(case_inputs,print_output=False)
    print(f'It would take {ytr} years to retire in a {case_type} Economy')

It would take 61 years to retire in a Recession Economy
It would take 28 years to retire in a Normal Economy
It would take 23 years to retire in a Expansion Economy


In [8]:
Case_probabilitires={
    'Recession':0.2,
    'Normal':0.5,
    'Expansion':0.3
}
expected_ytr=0
for case_type,case_inputs in cases.items():
    ytr=years_to_retirement(case_inputs,print_output=False)
    weighted_ytr=ytr*Case_probabilitires[case_type]
    expected_ytr=expected_ytr+weighted_ytr
print(
    f'It would take { expected_ytr:.0f} years to retire considering'
    f' a {Case_probabilitires["Recession"]:.0%} chance  of a recession and'
    f' an {Case_probabilitires["Expansion"]:.0%} chance  of a expansionary economy'  
)

It would take 33 years to retire considering a 20% chance  of a recession and an 30% chance  of a expansionary economy
